# <div style=" text-align: center; font-weight: bold">Phase 02: Preprocessing data</div>

This is the preprocessing phase for data of the real estates for sale.

## Import necessary Python modules

In [142]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

## **Explore the data**

### Read the data from file:

In [143]:
real_estate_for_sale_df = pd.read_csv('../Data/real_estate_for_sale.csv')
real_estate_for_sale_df.head()

,Address,Type,Area,Price,Bedroom,Toilet,Floor,Furniture,Direction,Legal,Posting date,Expiry date,Ad type,Ad code
0,"Dự án Zenity, Đường Võ Văn Kiệt, Phường Cầu Kh...",Căn hộ chung cư,"161,08 m²","17,98 tỷ",3 phòng,3 phòng,NaN,Đầy đủ,Đông - Bắc,Sổ đỏ/ Sổ hồng,06/12/2023,13/12/2023,Tin VIP Kim Cương,38720367
1,"Dự án Zenity, Đường Võ Văn Kiệt, Phường Cầu Kh...",Căn hộ chung cư,116 m²,"9,8 tỷ",3 phòng,2 phòng,NaN,Đầy đủ,Đông - Bắc,Sổ đỏ/ Sổ hồng,03/12/2023,10/12/2023,Tin thường,38693652
2,"Dự án Zenity, Đường Võ Văn Kiệt, Phường Cầu Kh...",Căn hộ chung cư,77 m²,6 tỷ,2 phòng,2 phòng,NaN,Đầy đủ.,Đông - Nam,Sổ đỏ/ Sổ hồng,01/12/2023,11/12/2023,Tin VIP Kim Cương,38481731
3,"Dự án Lumiere Riverside, Đường Xa Lộ Hà Nội, P...",Căn hộ chung cư,76 m²,"6,2 tỷ",2 phòng,2 phòng,NaN,Đầy đủ,NaN,Hợp đồng mua bán,05/12/2023,12/12/2023,Tin VIP Kim Cương,38393009
4,"Dự án Zenity, Đường Võ Văn Kiệt, Phường Cầu Kh...",Căn hộ chung cư,95 m²,80 triệu/m²,2 phòng,2 phòng,NaN,Đầy đủ.,NaN,Sổ đỏ/ Sổ hồng.,07/12/2023,14/12/2023,Tin VIP Kim Cương,38734339


#### Num of rows and columns:

In [144]:
num_rows, num_cols = real_estate_for_sale_df.shape

print(f'Num of rows:  {num_rows}')
print (f'Num of columns:  {num_cols}')

Num of rows:  55312
Num of columns:  14


#### The meaning of each line. Does it matter if a line have different meaning?

The data is collected by crawling raw data from the website https://batdongsan.com.vn/    
Each line is the record of a advertisement of real estate. So there isn't any line that has different meaning.

#### Num of duplicated rows:

In [145]:
duplicate = real_estate_for_sale_df.duplicated().sum()

print (f' Nums of duplicated rows: {duplicate}')

 Nums of duplicated rows: 266


we can see there are duplicated rows in the dataset. The reason here is there are some advertisements is reposted in the website, result in the duplicated data.
So, we will drop these duplicated rows.

In [146]:
real_estate_for_sale_df.drop_duplicates(inplace= True)
real_estate_for_sale_df = real_estate_for_sale_df.reset_index(drop=True)

#### Ratio of missing values for each column:

In [147]:
def missing_ratio(column):
    missing_values = column.isnull().sum()
    total_values = len(column)
    return (missing_values / total_values) * 100

missing_ratios_df = real_estate_for_sale_df.agg(missing_ratio).to_frame()
missing_ratios_df.columns = ['Missing ratio']

missing_ratios_df

,Missing ratio
Address,0.000000
Type,0.000000
Area,0.036333
Price,0.000000
Bedroom,38.362460
Toilet,42.279185
Floor,54.421756
Furniture,58.511063
Direction,78.234567
Legal,31.195727


- We can see that the fields `Address`, `Type`, `Price`, `Posting date`, `Expiry date`, `Ad type`, `Ad code` have no missing values. This is easy to understand the reasons: The `Address`, `Type` and `Price` is the basic data that a post have to contain.
-  The other fields like `Bedroom`, `Toilet`, `Floor`, `Furniture` are more special. The dataset contain many types of real easte, some of them are `Đất nền dự án`, `Đất bán`, `Trang trại, khu nghỉ dưỡng`, these types of real easte will not have the information of above fields. So reasonly, the fields will be lacked. We will consider it more clearly in the next part.
- All the rests is have a large ratio of missing value. from *31.176588 %* in `Legal` up to *78.23849 %* in `Direction`. So the data preprocessing of data is so necessary before we make the analysis.

#### **The meaning of each columns**
Let's see all the columns of the dataset.

In [148]:
real_estate_for_sale_df.columns.to_list()

['Address',
 'Type',
 'Area',
 'Price',
 'Bedroom',
 'Toilet',
 'Floor',
 'Furniture',
 'Direction',
 'Legal',
 'Posting date',
 'Expiry date',
 'Ad type',
 'Ad code']

- `Address`: address of the real estate. With the real estate is apartment, the address can also contain the Project of the real estate.
- `Type` : the category of the real estate.
- `Area`: the area of the real estate.
- `Price`: the price of the real estate. It can be the total price or just the price per m^2.
- `Bedroom`: number of bedroom in the real estate.
- `Toilet`: number of toilet in the real estate.
- `Floor`: number of floor.
- `Furniture`: the furniture status of the real estate.
- `Direction`: the direction of the real estate.
- `Legal`: Some legal policy of the real estate.
- `Posting date`: The day that the advertisement was posted.
- `Expiry date`: the day the real estate was enable.
- `Ad type`: the type of advertisement.
- `Ad code`: the code of advertisement.

#### Data type of each colmuns:

In [149]:
cols_type = real_estate_for_sale_df.dtypes
cols_type

Address         object
Type            object
Area            object
Price           object
Bedroom         object
Toilet          object
Floor           object
Furniture       object
Direction       object
Legal           object
Posting date    object
Expiry date     object
Ad type         object
Ad code          int64
dtype: object

- Nearly all of the columns is in Object type. These columns will not suitable for the further analysis. So we need to do some preprocessing on the data types.

### Some preprocessing:

- **Some obsevation:**
    - `Area`, `Price`, `Bedroom`, `Toilet` and `Floor` should be numerical columns, so we will convert them into numeric data types.

### Preprocessing for numeric columns:
    

#### **1. Area:**
First, we will find that if the values is in the same unit.

In [150]:
area_values = real_estate_for_sale_df[real_estate_for_sale_df['Area'].notna()]['Area'].to_list()

unit_list = []
value_list = []
for area in area_values:
    unit_list.append(area.split(' ')[1])
    value_list.append(area.split(' ')[0])

set(unit_list)


{'m²'}

So we can see that all the values in columns `Area` is in the same unit. Now we just set the `Area` columns with the new value, change the data type, then rename it for a clearly meaning.

In [151]:
#77,5 , 5.629,2

nonnan_indices = real_estate_for_sale_df['Area'].notna()
real_estate_for_sale_df.loc[nonnan_indices, 'Area'] = value_list
area_list = real_estate_for_sale_df['Area'].dropna().to_list()
cleaned_area_list = []

for area in area_list:
    if (',' in area) and ('.' in area):
        cleaned_area = area.replace('.', '').replace(',', '.')
    elif ',' in area:
        cleaned_area = area.replace(',', '.')
    else:
        cleaned_area = area

    if '.' in area:
        count = len(area.split('.')[-1])
        if count == 3:
            cleaned_area = area.replace('.','')
        

    cleaned_area_list.append(cleaned_area)

real_estate_for_sale_df.loc[nonnan_indices, 'Area'] = cleaned_area_list

real_estate_for_sale_df['Area'] = real_estate_for_sale_df['Area'].astype('float64')
real_estate_for_sale_df.rename(columns={'Area': 'Area(m2)'}, inplace=True)

#### **Price:**
First, we will find that if the values is in the same unit.

In [152]:
price_values = real_estate_for_sale_df['Price'].to_list()

unit_list = []
value_list = []
for price in price_values:
    unit_list.append(price.split(' ')[1])
    value_list.append(price.split(' ')[0])

set(unit_list)

{'nghìn/m²', 'thuận', 'triệu', 'triệu/m²', 'tỷ', 'tỷ/m²'}

The `Price` column contains various units, so we will convert them to the *tỷ* unit

In [153]:
price_list = real_estate_for_sale_df['Price'].to_list()

new_prices = []
for index, value in enumerate(price_list):
    if 'nghìn/m²' in value:
        area = real_estate_for_sale_df.loc[index, 'Area(m2)']
        price = str(value).split(' ')[0]

        price = (float(price) * area )/1000000
        new_prices.append((index, price))

    elif 'triệu' in value:
        price, unit = value.split(' ')
        if unit == 'triệu':
            price = price.replace(',', '.')
            price = float(price) / 1000
            new_prices.append((index, price))
        
        elif unit == 'triệu/m²':
            area = real_estate_for_sale_df.loc[index, 'Area(m2)']
            price = price.replace(',', '.')
            price = (float(price) * area) / 1000
            new_prices.append((index, price))
    elif 'tỷ' in value:
        price, unit = value.split(' ')
        if unit == 'tỷ/m²':
            area = real_estate_for_sale_df.loc[index, 'Area(m2)']
            price = price.replace(',', '.')
            price = (float(price) * area)
            new_prices.append((index, price))
        else:
            price = price.replace(',', '.')
            new_prices.append((index, float(price)))

price_column = np.array(price_list)
for index, price in new_prices:
    price_column[index] = price

real_estate_for_sale_df['Price'] = price_column

real_estate_for_sale_df.rename(columns={'Price': 'Price(tỷ)'}, inplace=True)

In [154]:
real_estate_for_sale_df

,Address,Type,Area(m2),Price(tỷ),Bedroom,Toilet,Floor,Furniture,Direction,Legal,Posting date,Expiry date,Ad type,Ad code
0,"Dự án Zenity, Đường Võ Văn Kiệt, Phường Cầu Kh...",Căn hộ chung cư,161.08,17.98,3 phòng,3 phòng,NaN,Đầy đủ,Đông - Bắc,Sổ đỏ/ Sổ hồng,06/12/2023,13/12/2023,Tin VIP Kim Cương,38720367
1,"Dự án Zenity, Đường Võ Văn Kiệt, Phường Cầu Kh...",Căn hộ chung cư,116.00,9.8,3 phòng,2 phòng,NaN,Đầy đủ,Đông - Bắc,Sổ đỏ/ Sổ hồng,03/12/2023,10/12/2023,Tin thường,38693652
2,"Dự án Zenity, Đường Võ Văn Kiệt, Phường Cầu Kh...",Căn hộ chung cư,77.00,6.0,2 phòng,2 phòng,NaN,Đầy đủ.,Đông - Nam,Sổ đỏ/ Sổ hồng,01/12/2023,11/12/2023,Tin VIP Kim Cương,38481731
3,"Dự án Lumiere Riverside, Đường Xa Lộ Hà Nội, P...",Căn hộ chung cư,76.00,6.2,2 phòng,2 phòng,NaN,Đầy đủ,NaN,Hợp đồng mua bán,05/12/2023,12/12/2023,Tin VIP Kim Cương,38393009
4,"Dự án Zenity, Đường Võ Văn Kiệt, Phường Cầu Kh...",Căn hộ chung cư,95.00,7.6,2 phòng,2 phòng,NaN,Đầy đủ.,NaN,Sổ đỏ/ Sổ hồng.,07/12/2023,14/12/2023,Tin VIP Kim Cương,38734339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55041,"Đường Nguyễn Sáng, Phường Tây Thạnh, Tân Phú, ...",Nhà mặt phố,144.00,16.5,40 phòng,42 phòng,7 tầng,Đầy đủ,NaN,Sổ đỏ/ Sổ hồng,08/11/2023,08/12/2023,Tin thường,38500057
55042,"Dự án KĐT Vạn Phúc City, Đường Nguyễn Thị Nhun...",Nhà mặt phố,140.00,34.0,5 phòng,6 phòng,6 tầng,NaN,NaN,Sổ đỏ/ Sổ hồng.,09/12/2023,08/01/2024,Tin thường,38016583
55043,"Phường 8, Gò Vấp, Hồ Chí Minh",Nhà mặt phố,66.00,6.7,4 phòng,6 phòng,3 tầng,Đầy đủ cơ bản,NaN,Đã có sổ đất ở đô thị sở hữu lâu dài,08/11/2023,08/12/2023,Tin thường,38500616
55044,"Đường Nguyễn Văn Lượng, Phường 17, Gò Vấp, Hồ ...",Nhà riêng,63.00,5.7,1 phòng,1 phòng,1 tầng,Không nội thất,Nam,Sổ đỏ/ Sổ hồng,08/11/2023,08/12/2023,Tin thường,38499922


#### **Bedroom:**

In [155]:

bedroom_mask = real_estate_for_sale_df['Bedroom'].notna()
bedroom_values = real_estate_for_sale_df.loc[bedroom_mask, 'Bedroom']

unit_list = []
value_list = []
for bedroom in bedroom_values:
    unit_list.append(bedroom.split(' ')[1])
    value_list.append(bedroom.split(' ')[0])

set(unit_list)


{'phòng'}

In [156]:
real_estate_for_sale_df.loc[bedroom_mask, 'Bedroom'] = value_list
real_estate_for_sale_df['Bedroom'] = real_estate_for_sale_df['Bedroom'].astype('float64')

#### **Toilet:**

In [157]:
toilet_mask = real_estate_for_sale_df['Toilet'].notna()

toilet_values = real_estate_for_sale_df.loc[toilet_mask, 'Toilet']

unit_list = []
value_list = []
for toilet in toilet_values:
    unit_list.append(toilet.split(' ')[1])
    value_list.append(toilet.split(' ')[0])

set(unit_list)

{'phòng'}

In [158]:
real_estate_for_sale_df.loc[toilet_mask, 'Toilet'] = value_list
real_estate_for_sale_df['Toilet'] = real_estate_for_sale_df['Toilet'].astype('float64')

#### **Floor:**

In [159]:
floor_mask = real_estate_for_sale_df['Floor'].notna()

floor_values = real_estate_for_sale_df.loc[floor_mask, 'Floor']

unit_list = []
value_list = []
for floor in floor_values:
    unit_list.append(floor.split(' ')[1])
    value_list.append(floor.split(' ')[0])

set(unit_list)

{'tầng'}

In [160]:
real_estate_for_sale_df.loc[floor_mask, 'Floor'] = value_list
real_estate_for_sale_df['Floor'] = real_estate_for_sale_df['Floor'].astype('float64')

### Preprocessing for categorical columns:
#### **Addresss:**

- With the `Address` column, there is some thing we can dicuss here:
    - The fully address is does not really meaningfull for my analysis. The base idea is that all the real estate is located in Ho Chi Minh City, so we can extract the district of the real estate.
    - In many real estates, the address also contain the project they belong to. So we could also extract the project for further.

##### Extract the district of real estates:

In [161]:
address_df = real_estate_for_sale_df['Address'].values

print(type(address_df))

district_list = []
unformal_address = []
for i, address in enumerate(address_df):
    split = address.split(',')
    district = ""
    if 'Hồ Chí Minh' in split[-1] or 'TP.HCM' in split[-1]:
        try:
            district = split[-2].strip()
        except: 
            split = address.split(' ')
            district = split[-5] + " " + split[-4]
   
        prefix_list = ['quận', 'huyện', 'thành phố', 'q.', 'TP.']
        for prefix in prefix_list:
            if district.lower().strip().startswith(prefix.lower()):
                district = district[len(prefix):].strip()
   
          
    else:
        district = "Không"
        unformal_address .append((i, address))

    if 'Phường' in district or len(district) == 0:
        district = "Không"
    district_list.append(district)


<class 'numpy.ndarray'>


##### Update value of the column

In [162]:

real_estate_for_sale_df['District'] = district_list

real_estate_for_sale_df = real_estate_for_sale_df[real_estate_for_sale_df['District'] != 'Không']

real_estate_for_sale_df['District'].value_counts()


District
7             5243
2             5110
Bình Thạnh    4065
9             4021
Tân Bình      3522
Thủ Đức       3131
Gò Vấp        2964
Tân Phú       2934
1             2777
Bình Tân      2515
Phú Nhuận     2259
3             2251
12            2230
10            2131
Bình Chánh    1915
Nhà Bè        1707
8             1236
Củ Chi        1014
4              955
6              818
5              744
11             696
Hóc Môn        653
Cần Giờ        141
Name: count, dtype: int64

#### **Furniture:**

With the `Furniture` columns, the furniture status is given and describled by the customers. So there are many type of the furnitures. But we will use some simple status like `Không nội thất`, `Cơ bản`, `Đầy đủ`, `Cao cấp`, `Khác`. We will categorize all the values of this column to these types.      


we will try to

In [163]:
real_estate_for_sale_df['Furniture'].value_counts()

Furniture
Đầy đủ                              11702
Cơ bản                               4759
Đầy đủ.                              1591
Không nội thất                        733
Cơ bản.                               587
                                    ...  
Tặng nội thất dính tường                1
Nội thất CĐT cao cấp.                   1
253AHS                                  1
Nội thất căn hộ đầy đủ, cao cấp.        1
Ko                                      1
Name: count, Length: 1236, dtype: int64

In [166]:
furniture_masked = real_estate_for_sale_df['Furniture'].notna()
furniture_df = real_estate_for_sale_df.loc[furniture_masked, 'Furniture']

cleaned_furniture_list = []
for i, furniture in enumerate(furniture_df):

    furniture = furniture.lower().strip()
    if ("cao cấp" in furniture or "sang trọng" in furniture):
        furniture = 'Nội thất cao cấp'
    elif ("đủ" in furniture or "full" in  furniture):
        furniture = 'Nội thất đầy đủ'
    elif ("cơ bản" in furniture or 'ít' in furniture):
        furniture = 'Nội thất cơ bản'
    elif ("không" in furniture or 'thô' in furniture):
        furniture = "Không nội thất"
    else:
        furniture = 'Khác'
    cleaned_furniture_list.append(furniture)

print(cleaned_furniture_list)

['Nội thất đầy đủ', 'Nội thất đầy đủ', 'Nội thất đầy đủ', 'Nội thất đầy đủ', 'Nội thất đầy đủ', 'Nội thất cao cấp', 'Nội thất cơ bản', 'Nội thất cơ bản', 'Nội thất đầy đủ', 'Nội thất đầy đủ', 'Nội thất cơ bản', 'Nội thất cơ bản', 'Nội thất đầy đủ', 'Nội thất cơ bản', 'Nội thất cơ bản', 'Nội thất cơ bản', 'Nội thất cơ bản', 'Nội thất cơ bản', 'Nội thất đầy đủ', 'Nội thất cơ bản', 'Nội thất cao cấp', 'Nội thất đầy đủ', 'Nội thất đầy đủ', 'Nội thất đầy đủ', 'Nội thất cơ bản', 'Nội thất cơ bản', 'Không nội thất', 'Nội thất đầy đủ', 'Nội thất đầy đủ', 'Nội thất cơ bản', 'Nội thất đầy đủ', 'Nội thất cơ bản', 'Nội thất đầy đủ', 'Không nội thất', 'Nội thất cơ bản', 'Nội thất cơ bản', 'Nội thất đầy đủ', 'Nội thất đầy đủ', 'Nội thất đầy đủ', 'Nội thất đầy đủ', 'Nội thất đầy đủ', 'Khác', 'Nội thất cao cấp', 'Nội thất cơ bản', 'Nội thất cơ bản', 'Nội thất đầy đủ', 'Nội thất đầy đủ', 'Nội thất đầy đủ', 'Nội thất đầy đủ', 'Nội thất đầy đủ', 'Nội thất cơ bản', 'Nội thất cơ bản', 'Nội thất đầy đủ', 'N

In [167]:
# raise Exception('NotImplemented')
real_estate_for_sale_df.loc[furniture_masked, 'Furniture'] = cleaned_furniture_list

real_estate_for_sale_df['Furniture'].value_counts()


Furniture
Nội thất đầy đủ     14509
Nội thất cơ bản      5611
Nội thất cao cấp     1061
Không nội thất       1006
Khác                  644
Name: count, dtype: int64

#### **Legal:**

With the `Legal` column, there will be categories below:
- `Sổ đỏ và Sổ hồng`
- `Sổ hồng`
- `Sổ đỏ`
- `Hợp đồng mua bán`
- `Đang chờ sổ`
- `Khác`

In [181]:
legal_masked = real_estate_for_sale_df['Legal'].notna()
legal_df = real_estate_for_sale_df.loc[legal_masked, 'Legal']

legal_list = []
for legal in legal_df:
    legal = legal.lower().strip()
    if 'sổ đỏ/ sổ hồng' in legal or 'có sổ' in legal or 'đầy đủ' in legal:
        legal = 'Sổ đỏ và sổ hồng'
    elif 'sổ đỏ' in legal:
        legal = 'Sổ đỏ'
    elif 'sổ hồng' in legal or 'shr' in legal:
        legal = 'Sổ hồng'
    elif 'hợp đồng mua bán' in legal or 'hđmb' in legal:
        legal = 'Hợp đồng mua bán'
    elif 'đang chờ sổ' in legal:
        legal = 'Đang chờ sổ'
    else:
        legal = 'Khác'
    legal_list.append(legal)
    

In [182]:
real_estate_for_sale_df.loc[legal_masked, 'Legal'] = legal_list

real_estate_for_sale_df['Legal'].value_counts()


Legal
Sổ đỏ và sổ hồng    30284
Hợp đồng mua bán     3524
Sổ hồng              2780
Đang chờ sổ           615
Khác                  396
Sổ đỏ                 264
Name: count, dtype: int64